In [4]:
import torch
import torch_geometric

import pandas as pd
import numpy as np

import rdkit
import rdkit.Chem
import rdkit.Chem.AllChem
from rdkit import Chem

from tqdm import tqdm
from copy import deepcopy
import random
import re
import os
import shutil
import argparse
import sys
import ast

from datasets.dataset_3D import *
from models.dimenetpp import *


In [5]:
model_dictionary = {
    ('amine', 'atom', 'NBO_LP_energy', 'Boltz'): 'trained_models/combined_amines/NBO_LP_energy/boltz/model_best.pt',
    ('amine', 'atom', 'NBO_LP_energy', 'max'):   'trained_models/combined_amines/NBO_LP_energy/max/model_best.pt',
    ('amine', 'atom', 'NBO_LP_energy', 'min'):   'trained_models/combined_amines/NBO_LP_energy/min/model_best.pt',
    ('amine', 'atom', 'NBO_LP_energy', 'lowE'): 'trained_models/combined_amines/NBO_LP_energy/min_E/model_best.pt',
    
    ('amine', 'atom', 'NBO_LP_occupancy', 'Boltz'): 'trained_models/combined_amines/NBO_LP_occupancy/boltz/model_best.pt',
    ('amine', 'atom', 'NBO_LP_occupancy', 'max'):   'trained_models/combined_amines/NBO_LP_occupancy/max/model_best.pt',
    ('amine', 'atom', 'NBO_LP_occupancy', 'min'):   'trained_models/combined_amines/NBO_LP_occupancy/min/model_best.pt',
    ('amine', 'atom', 'NBO_LP_occupancy', 'lowE'): 'trained_models/combined_amines/NBO_LP_occupancy/min_E/model_best.pt',
    
    ('amine', 'atom', 'pyr_agranat', 'Boltz'): 'trained_models/combined_amines/pyr_agranat/boltz/model_best.pt',
    ('amine', 'atom', 'pyr_agranat', 'max'):   'trained_models/combined_amines/pyr_agranat/max/model_best.pt',
    ('amine', 'atom', 'pyr_agranat', 'min'):   'trained_models/combined_amines/pyr_agranat/min/model_best.pt',
    ('amine', 'atom', 'pyr_agranat', 'lowE'): 'trained_models/combined_amines/pyr_agranat/min_E/model_best.pt',
    
    ('amine', 'atom', 'Vbur', 'Boltz'): 'trained_models/combined_amines/Vbur/boltz/model_best.pt',
    ('amine', 'atom', 'Vbur', 'max'):   'trained_models/combined_amines/Vbur/max/model_best.pt',
    ('amine', 'atom', 'Vbur', 'min'):   'trained_models/combined_amines/Vbur/min/model_best.pt',
    ('amine', 'atom', 'Vbur', 'lowE'): 'trained_models/combined_amines/Vbur/min_E/model_best.pt',
    
    ('amine', 'bond', 'Sterimol_B1', 'Boltz'): 'trained_models/amines/Sterimol_B1/boltz/model_best.pt',
    ('amine', 'bond', 'Sterimol_B1', 'max'):   'trained_models/amines/Sterimol_B1/max/model_best.pt',
    ('amine', 'bond', 'Sterimol_B1', 'min'):   'trained_models/amines/Sterimol_B1/min/model_best.pt',
    ('amine', 'bond', 'Sterimol_B1', 'lowE'): 'trained_models/amines/Sterimol_B1/min_E/model_best.pt',
    
    ('amine', 'bond', 'Sterimol_B5', 'Boltz'): 'trained_models/amines/Sterimol_B5/boltz/model_best.pt',
    ('amine', 'bond', 'Sterimol_B5', 'max'):   'trained_models/amines/Sterimol_B5/max/model_best.pt',
    ('amine', 'bond', 'Sterimol_B5', 'min'):   'trained_models/amines/Sterimol_B5/min/model_best.pt',
    ('amine', 'bond', 'Sterimol_B5', 'lowE'): 'trained_models/amines/Sterimol_B5/min_E/model_best.pt',
    
    ('amine', 'bond', 'Sterimol_L', 'Boltz'): 'trained_models/amines/Sterimol_L/boltz/model_best.pt',
    ('amine', 'bond', 'Sterimol_L', 'max'):   'trained_models/amines/Sterimol_L/max/model_best.pt',
    ('amine', 'bond', 'Sterimol_L', 'min'):   'trained_models/amines/Sterimol_L/min/model_best.pt',
    ('amine', 'bond', 'Sterimol_L', 'lowE'): 'trained_models/amines/Sterimol_L/min_E/model_best.pt',

    ('amine', 'mol', 'dipole', 'Boltz'): 'trained_models/combined_amines/dipole/boltz/model_best.pt',
    ('amine', 'mol', 'dipole', 'max'):   'trained_models/combined_amines/dipole/max/model_best.pt',
    ('amine', 'mol', 'dipole', 'min'):   'trained_models/combined_amines/dipole/min/model_best.pt',
    ('amine', 'mol', 'dipole', 'lowE'): 'trained_models/combined_amines/dipole/min_E/model_best.pt',
    
}

In [6]:
num_workers = 4
use_atom_features = 1

conformer_data_file = 'data/3D_model_primaryamine_rdkit_conformers.csv'
conformers_df = pd.read_csv(conformer_data_file).reset_index(drop = True)
conformers_df['mols'] = [rdkit.Chem.MolFromMolBlock(m, removeHs = False) for m in conformers_df.mol_block]
conformers_df['mols_noHs'] = [rdkit.Chem.RemoveHs(m) for m in conformers_df['mols']]


In [ ]:
# testing loop definition

def loop(model, batch, property_type = 'bond'):
    
    batch = batch.to(device)
        
    if property_type == 'bond':
        out = model(
            batch.x.squeeze(), 
            batch.pos, 
            batch.batch,
            batch.atom_features,
            select_bond_start_atom_index = batch.bond_start_ID_index,
            select_bond_end_atom_index = batch.bond_end_ID_index,
        )
    
    elif property_type == 'atom':
        out = model(
            batch.x.squeeze(),
            batch.pos, 
            batch.batch,
            batch.atom_features,
            select_atom_index = batch.atom_ID_index,
        )
        
    elif property_type == 'mol':
        out = model(
            batch.x.squeeze(),
            batch.pos,
            batch.batch,
            batch.atom_features,
        )
    
    targets = batch.targets
    pred_targets = out[0].squeeze()
    mse_loss = torch.mean(torch.square(targets - pred_targets))
    mae = torch.mean(torch.abs(targets - pred_targets))    
    
    return targets.detach().cpu().numpy(), pred_targets.detach().cpu().numpy()


In [7]:
for model_selection in model_dictionary:
    
    keep_explicit_hydrogens = model_selection[2] in ['H5_NBO_charge', 'H5_NMR_shift', 'NBO_charge_H_avg', 'NBO_charge_H_min', 'NMR_shift_H_avg', 'NBO_charge_H', 'NMR_shift_H']
    remove_Hs_except_functional = 2 if keep_explicit_hydrogens else False
    
    mol_type = model_selection[0]
    property_type = model_selection[1]
    prop =  model_selection[2]
    agg = model_selection[3]
    
    if mol_type == 'acid':
        descriptor_data_file = f'data/acid/{property_type}/{prop}_{agg}.csv'
    if mol_type == 'amine':
        try:
            descriptor_data_file = f'data/combined_amine/{property_type}/{prop}_{agg}.csv'
            descriptor_df = pd.read_csv(descriptor_data_file, converters={"bond_atom_tuple": ast.literal_eval})
        except:
            descriptor_data_file = f'data/primary_amine/{property_type}/{prop}_{agg}.csv'
            descriptor_df = pd.read_csv(descriptor_data_file, converters={"bond_atom_tuple": ast.literal_eval})
    if mol_type == 'sec_amine':
        try:
            descriptor_data_file = f'data/combined_amine/{property_type}/{prop}_{agg}.csv'
            descriptor_df = pd.read_csv(descriptor_data_file, converters={"bond_atom_tuple": ast.literal_eval})
        except:
            descriptor_data_file = f'data/secondary_amine/{property_type}/{prop}_{agg}.csv'
            descriptor_df = pd.read_csv(descriptor_data_file, converters={"bond_atom_tuple": ast.literal_eval})
        
    
    pretrained_model = model_dictionary[model_selection]
    
    # -------------------------------------------
    # Loading training data (regression targets and input conformers)
    
    descriptor_df = pd.read_csv(descriptor_data_file, converters={"bond_atom_tuple": ast.literal_eval})
    
    merged_df = conformers_df.merge(descriptor_df, on = 'Name_int')
    test_dataframe = merged_df[merged_df.split == 'test'].reset_index(drop = True)
        
    # -------------------------------------------
    # creating model, optimizer, dataloaders
    
    device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if property_type == 'atom':
        atom_ID_test = np.array(list(test_dataframe.atom_index), dtype = int)
    else:
        atom_ID_test = None
    
    if property_type == 'bond':
        bond_ID_1_test = np.array(list(test_dataframe.bond_atom_tuple), dtype = int)[:, 0]
        bond_ID_2_test = np.array(list(test_dataframe.bond_atom_tuple), dtype = int)[:, 1]
    else:
        bond_ID_1_test = None
        bond_ID_2_test = None
    
    if keep_explicit_hydrogens:
        mols_test = list(test_dataframe.mols)
    else:
        mols_test = list(test_dataframe.mols_noHs)
    
    test_dataset = Dataset_3D(
        property_type = property_type,
        mols = mols_test, 
        mol_types = list(test_dataframe['mol_type']),
        targets = list(test_dataframe['y']),
        ligand_ID = np.array(test_dataframe['Name_int']),
        atom_ID = atom_ID_test,
        bond_ID_1 = bond_ID_1_test,
        bond_ID_2 = bond_ID_2_test,
        remove_Hs_except_functional = remove_Hs_except_functional,
    )
    test_loader = torch_geometric.loader.DataLoader(
        dataset = test_dataset,
        batch_size = 100,
        shuffle = False,
        num_workers = num_workers,
    )
    
    example_data = test_dataset[0]
    atom_feature_dim = int(example_data.atom_features.shape[-1]) # 53
    
    model = DimeNetPlusPlus(
        property_type = property_type, 
        use_atom_features = use_atom_features, 
        atom_feature_dim = atom_feature_dim if use_atom_features else 1, 
    )
    if pretrained_model != '':
        model.load_state_dict(torch.load(pretrained_model, map_location=next(model.parameters()).device), strict = True)
    
    model.to(device)
    
    # -------------------------------------------
    # testing
    
    model.eval()
    test_targets = []
    test_pred_targets = []
    for batch in tqdm(test_loader):
        with torch.no_grad():
            target, pred_target = loop(
                model, 
                batch, 
                property_type = property_type, 
            )
        test_targets.append(target)
        test_pred_targets.append(pred_target)
        
    test_targets = np.concatenate(test_targets)
    test_pred_targets = np.concatenate(test_pred_targets)
    
    test_results = pd.DataFrame()
    test_results['Name_int'] = test_dataframe.Name_int
    test_results['targets'] = test_targets
    test_results['predictions'] = test_pred_targets
    
    test_results=test_results.groupby('Name_int').apply(lambda x: x.mean())
    
    #test_results.to_csv(f'test_set_predictions/{mol_type}_{property_type}_{prop}_{agg}.csv')
    
    test_MAE = np.mean(np.abs(np.array(test_results['targets']) - np.array(test_results['predictions'])))
    test_R2 = np.corrcoef(np.array(test_results['targets']), np.array(test_results['predictions']))[0][1] ** 2
    
    print(model_selection)
    print('    ','MAE:', test_MAE, 'R2:', test_R2)
    

100%|██████████| 49/49 [00:21<00:00,  2.32it/s]


('amine', 'atom', 'NBO_LP_energy', 'Boltz')
     MAE: 0.0021559334838921244 R2: 0.8744058369844313


100%|██████████| 49/49 [00:33<00:00,  1.47it/s]


('amine', 'atom', 'NBO_LP_energy', 'max')
     MAE: 0.002756369198382142 R2: 0.881383000730126


100%|██████████| 49/49 [00:58<00:00,  1.20s/it]


...

('amine', 'mol', 'dipole', 'max')
     MAE: 0.3140860234013936 R2: 0.8865966872100713


100%|██████████| 49/49 [00:45<00:00,  1.08it/s]


('amine', 'mol', 'dipole', 'min')
     MAE: 0.31920078408862895 R2: 0.7311354429918763


100%|██████████| 49/49 [00:45<00:00,  1.09it/s]


('amine', 'mol', 'dipole', 'lowE')
     MAE: 0.4911697904469996 R2: 0.6835344215960967
